In [4]:
%ls

LICENSE-CODE                deepseek_vl.egg-info/
LICENSE-MODEL               gemini_explanations.jsonl
Makefile                    gemini_explanations2.jsonl
README.md                   images/
cli_chat.py                 inference.py
data/                       pyproject.toml
deepseek_vl/                requirements.txt


In [2]:
%ls
%cd DeepSeek-VL/

1.nls             DeepSeek-VL-main/ distill.ipynb     eval.py
2.nls             dataset.py        distill.py        finetune.ipynb
DeepSeek-VL/      distill/          distill.zip       zeroshot.ipynb
/Users/User/Desktop/Deep learning/deepseek/DeepSeek-VL


/opt/anaconda3/envs/ml_hw3/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!pip install --upgrade google-generativeai aiohttp aiofiles tqdm nest_asyncio aiolimiter

  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.39.0-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached grpcio_status-1.71.0-py3-none-any.whl.metadata (1.1

In [2]:
# ==== requirements ====
# pip install --upgrade google-generativeai aiohttp aiofiles tqdm nest_asyncio

import os, json, asyncio, aiofiles
from pathlib import Path
from tqdm.asyncio import tqdm_asyncio
import google.generativeai as genai
import nest_asyncio
from pathlib import Path
nest_asyncio.apply()                         # for Jupyter / Colab


genai.configure(api_key="")  # replace with your API key
MODEL = genai.GenerativeModel("gemini-1.5-flash")

SAFETY = [{"category": "HATE_SPEECH", "threshold": "block_none"}]

PROMPT_TMPL = """\
You are an expert content-moderation analyst.

Image: <image>
Meme text: {text}
Known label: {label_str}

Explain, in 2-3 sentences, why the meme is {label_str}.
Write only the explanation; do not repeat the label.
"""

In [ ]:
import json
from tqdm import tqdm
import google.generativeai as genai  # pip install google-generativeai
from PIL import Image
from pathlib import Path
# ==== Configuration ====
GEMINI_API_KEY = ""  # <-- 🔥 Change this
dev_path = "data/dev.jsonl"
max_samples = 500  # Set to None for all data, or a number like 100 for quick testing

# ==== Initialize Gemini ====
genai.configure(api_key=GEMINI_API_KEY)
MODEL = genai.GenerativeModel("gemini-1.5-flash")

# ==== Load data ====
print("Loading data...")
examples = []
def load_jsonl(p): return [json.loads(l) for l in open(p, encoding="utf-8")]

examples  = load_jsonl("data/dev.jsonl")
"""ArithmeticError
with open(dev_path, 'r') as f:
    for line in f:
        examples.append(json.loads(line.strip()))
"""

if max_samples:
    examples = examples[:max_samples]
print(examples[0])
# ==== Evaluation ====
prompt_prefix = (
    "Determine whether this meme is hateful or not. "
    "First give a 1-2 sentence explanation, then on a new line write "
    "FINAL ANSWER: YES or NO.\n"
)

predictions = []
groundtruth = []

print(f"Running on {len(examples)} samples...")

for ex in tqdm(examples):
    image_path = ex["img"]  # example: img/08291.png
    text = ex["text"]
    label = ex["label"]  # 1 = hateful (YES), 0 = non-hateful (NO)

    # Prepare prompt
    full_prompt = prompt_prefix + f"Meme text: {text}"

    try:
        # Read image bytes
        with open(Path("data") / image_path, "rb") as f:
            image_bytes = f.read()

        # Send to Gemini
        response = MODEL.generate_content(
            contents=[
                {"text": full_prompt},
                {"mime_type": "image/png", "data": image_bytes},
            ]
        )
        output_text = response.text.strip()
    except Exception as e:
        print(f"⚠️  Error with {image_path}: {e}")
        predictions.append(-1)
        groundtruth.append(label)
        continue

    # Parse YES/NO
    #print(f"Output: {output_text}")
    if "YES" in output_text:
        predictions.append(1)
    elif "NO" in output_text:
        predictions.append(0)
    else:
        predictions.append(-1)  # invalid output

    groundtruth.append(label)

# ==== Compute Accuracy ====
correct = 0
total = 0
for pred, gt in zip(predictions, groundtruth):
    if pred == -1:
        continue
    if pred == gt:
        correct += 1
    total += 1

accuracy = correct / total if total > 0 else 0
print(f"\n✅ Gemini Rough Accuracy: {accuracy:.4f} ({correct}/{total})")


In [25]:
import os, json, asyncio, aiofiles, time
from pathlib import Path
from collections import deque
from PIL import Image
from tqdm import tqdm
from aiolimiter import AsyncLimiter
from google.api_core import exceptions as gexc
import google.generativeai as genai
# ───── local token bucket to respect 1 000 000 TPM ──────────────────
TOKENS_PER_MIN, WIN = 1_000_000, 60
recent, tok_sum = deque(), 0
def token_wait(toks):
    """Block until <toks> fit into rolling 60-s window."""
    global tok_sum
    now = time.monotonic()
    while recent and now - recent[0][0] > WIN:
        ts, v = recent.popleft(); tok_sum -= v
    if tok_sum + toks > TOKENS_PER_MIN:
        sleep = WIN - (now - recent[0][0]) if recent else WIN
        print(f"[TPM] sleeping {round(sleep,1)} s …")
        time.sleep(sleep)
        token_wait(toks); return
    recent.append((now, toks)); tok_sum += toks

# ───── async request-limiter 15 RPM ────────────────────────────────
req_limiter = AsyncLimiter(50, 30)          # requests / minute

# ───── helpers ─────────────────────────────────────────────────────
def load_jsonl(p): return [json.loads(l) for l in open(p, encoding="utf-8")]

def already_done(out_path: Path) -> set[int]:
    if not out_path.exists(): return set()
    with out_path.open() as f:
        return {json.loads(line)["id"] for line in f}

async def call_gemini(pil_img, prompt, retries=5):
    for attempt in range(1, retries+1):
        try:
            # rate-limit requests
            async with req_limiter:
                resp = await asyncio.to_thread(
                    MODEL.generate_content,
                    [pil_img, prompt],
                    stream=True,
                    safety_settings=SAFETY,
                )
            break
        except genai.types.BlockedPromptException as e:
            print(f"[BlockedPrompt] Skipping sample due to safety filters.")
            return "PROHIBITED_CONTENT"  # <-- safely skip this sample
        
        except gexc.ResourceExhausted as err:
            # wait for quota reset
            delay = getattr(err, "retry_delay", None)
            delay = delay.seconds if delay else 40
            print(f"[429] sleeping {delay}s …")
            await asyncio.sleep(delay)
    else:
        raise RuntimeError("Too many 429 retries")

    # collect streamed text + token count
    text_parts = []
    for chunk in resp:                                   # stream iterator
        if chunk.candidates and chunk.candidates[0].content.parts:
            text_parts.append(chunk.candidates[0].content.parts[0].text)

    if not text_parts:                                   # finish_reason 8
        print(f"[warning] no text returned for ID {sample['id']}")
    token_wait(resp.usage_metadata.total_token_count)
    return " ".join(text_parts).strip()

async def explain_one(sample, fout):
    label_str = "HATEFUL" if sample["label"] else "NOT HATEFUL"
    prompt    = PROMPT_TMPL.format(text=sample["text"], label_str=label_str)

    try:
        pil_img = Image.open(Path("data") / sample["img"])
    except Exception as e:
        print(f"[error] failed to open image {sample['img']}: {e}")
        reason = "IMAGE_LOAD_FAILED"
    else:
        try:
            reason = await call_gemini(pil_img, prompt)
        except genai.types.BlockedPromptException as e:
            print(f"[BlockedPrompt] ID {sample['id']} got blocked. Writing PROHIBITED_CONTENT.")
            reason = "PROHIBITED_CONTENT"
        except Exception as e:
            print(f"[error] ID {sample['id']} failed due to unknown error: {e}")
            reason = "CALL_FAILED"

    record = {**sample, "gemini_reason": reason}
    await fout.write(json.dumps(record, ensure_ascii=False) + "\n")
    await fout.flush()
    return record

# ───── main coroutine ──────────────────────────────────────────────
async def main():
    data      = load_jsonl("data/train.jsonl")
    out_path  = Path("gemini_explanations.jsonl")
    done_ids  = already_done(out_path)

    # keep order but skip completed
    pending   = [s for s in data if s["id"] not in done_ids]
    print(f"{len(done_ids)} already finished, {len(pending)} left")

    async with aiofiles.open(out_path, "a") as fout:  # append-mode
        # ── sanity-check first two new items ──
        for samp in pending[:2]:
            try:
                rec = await explain_one(samp, fout)
                print("\nID", rec["id"], "→", rec["gemini_reason"][:120], "…")
            except Exception as e:
                print(f"[error] ID {samp['id']} failed during initial test: {e}")

        # ── concurrent processing of remaining memes ──
        tasks = [explain_one(s, fout) for s in pending[2:]]
        for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            try:
                await coro
            except Exception as e:
                print(f"[error] Task failed: {e}")

if __name__ == "__main__":
    asyncio.run(main())

8484 already finished, 16 left
[BlockedPrompt] ID 50312 got blocked. Writing PROHIBITED_CONTENT.

ID 50312 → PROHIBITED_CONTENT …
[BlockedPrompt] ID 28507 got blocked. Writing PROHIBITED_CONTENT.

ID 28507 → PROHIBITED_CONTENT …


  0%|          | 0/14 [00:00<?, ?it/s]

[BlockedPrompt] ID 34897 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 98406 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 71834 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 47510 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 30256 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 59260 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 93502 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 13695 got blocked. Writing PROHIBITED_CONTENT.


 71%|███████▏  | 10/14 [00:02<00:00,  5.04it/s]

[BlockedPrompt] ID 35740 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 13679 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 12793 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 26543 got blocked. Writing PROHIBITED_CONTENT.
[BlockedPrompt] ID 52743 got blocked. Writing PROHIBITED_CONTENT.


100%|██████████| 14/14 [00:03<00:00,  4.23it/s]

[BlockedPrompt] ID 34295 got blocked. Writing PROHIBITED_CONTENT.
